In [1]:
from rich import print

In [2]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="google/gemini-2.5-flash-preview-05-20",
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)
print(llm)

ChatOpenAI(
    client=<openai.resources.chat.completions.completions.Completions object at 0x7f352da326f0>,
    async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f352dc34b00>,
    root_client=<openai.OpenAI object at 0x7f352e061e20>,
    root_async_client=<openai.AsyncOpenAI object at 0x7f352da8c5c0>,
    model_name='google/gemini-2.5-flash-preview-05-20',
    model_kwargs={},
    openai_api_key=SecretStr('**********'),
    openai_api_base='https://openrouter.ai/api/v1'
)

In [3]:
import base64
from pathlib import Path

import ffmpeg

SUPPORTED_IMAGE_FORMATS = {".png", ".jpeg", ".jpg", ".webp", ".heic", ".heif"}
SUPPORTED_AUDIO_FORMATS = {".wav", ".mp3", ".aiff", ".aac", ".ogg", ".flac"}


def encode_image(image_path: str | Path) -> str:
    """Convert image file to base64-encoded string.

    Args:
        image_path: Path to the input image file

    Returns:
        Base64-encoded image data as string

    Raises:
        ValueError: If image format is not supported
    """
    image_path = Path(image_path)

    if image_path.suffix.lower() not in SUPPORTED_IMAGE_FORMATS:
        raise ValueError(f"Unsupported image format: {image_path.suffix}. Supported formats: PNG, JPEG, WEBP, HEIC, HEIF")

    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def encode_audio(audio_path: str | Path) -> str:
    """Convert audio file to base64-encoded audio data.

    Handles supported Gemini audio formats directly (WAV, MP3, AIFF, AAC, OGG, FLAC),
    converts other formats to MP3 in memory using ffmpeg, then returns base64-encoded audio data.

    Args:
        audio_path: Path to the input audio file

    Returns:
        Base64-encoded audio data as string
    """
    audio_path = Path(audio_path)

    if audio_path.suffix.lower() in SUPPORTED_AUDIO_FORMATS:
        with open(audio_path, "rb") as audio_file:
            audio_data = audio_file.read()
    else:
        # Convert other formats to MP3 in memory
        try:
            out, err = ffmpeg.input(str(audio_path)).output("pipe:", format="mp3", acodec="libmp3lame", audio_bitrate="192k").run(capture_stdout=True, capture_stderr=True)
            audio_data = out
        except ffmpeg.Error as e:
            raise RuntimeError(f"FFmpeg conversion failed: {e.stderr.decode() if e.stderr else str(e)}") from e

    # Encode to base64
    return base64.b64encode(audio_data).decode("utf-8")

In [4]:
import base64

# def encode_audio(audio_path):
#     with open(audio_path, "rb") as audio_file:
#         return base64.b64encode(audio_file.read()).decode("utf-8")


image_path = "image/150920049.png"
image_base64 = encode_image(image_path)

audio_path = "audio/sXoSILzgFug.m4a"
audio_base64 = encode_audio(audio_path)

In [ ]:
import os

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()


client = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)


response = client.chat.completions.create(
    model="gemini-2.5-flash-preview-05-20",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What do you see and hear?"},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
                {"type": "input_audio", "input_audio": {"data": audio_base64, "format": "mp3"}},
            ],
        },
    ],
)

print(response.choices[0].message.content)

In [36]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="gemini-2.5-flash-preview-05-20",
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
messages = [
    HumanMessage(
        content=[
            {"type": "text", "text": "What do you see and hear?"},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
            {"type": "input_audio", "input_audio": {"data": audio_base64, "format": "mp3"}},
        ]
    )
]

response = llm.invoke(messages)

print(response.content)

**What I see:**

I see a circular sticker with a teal or turquoise green background. On the sticker, there's a black silhouette of a
cartoon sloth's head. The sloth has a friendly, smiling expression with small, round eyes and a little curved 
mouth. Its head is somewhat shaggy, indicated by jagged lines around its outline. The bottom right corner of the 
sticker is peeled up, revealing a metallic silver adhesive backing underneath, and there's a subtle shadow cast by 
the sticker on a white surface.

**What I hear:**

I hear a male voice speaking in **Cantonese**. The speaker narrates at a steady pace, discussing symptoms and 
psychological aspects related to what is called "Degenerate Gambling Syndrome" or "Stock Speculation Syndrome."

The content can be summarized as:

*   It starts by listing symptoms like shortness of breath, insomnia, restlessness, or worsening temper, suggesting
these might indicate the syndrome.
*   It then poses a question about why people might make money initially when playing with small amounts in stock 
speculation but lose control when betting big.
*   The speaker attributes this behavior to being too nervous or worried, leading to quick reactions when winning 
but an inability to cut losses.
*   The narration concludes by stating that this is entirely related to one's mindset and that the current video 
will conduct a test.